# РК 2
## ИУ5-63Б Рассказов Н. Д.
## Вариант 16
## Датасет - SF Restaurant Scores - LIVES Standard

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import LabelEncoder

In [2]:
df = pd.read_csv('restaurant-scores-lives-standard.csv')

In [4]:
df.columns

Index(['business_id', 'business_name', 'business_address', 'business_city',
       'business_state', 'business_postal_code', 'business_latitude',
       'business_longitude', 'business_location', 'business_phone_number',
       'inspection_id', 'inspection_date', 'inspection_score',
       'inspection_type', 'violation_id', 'violation_description',
       'risk_category', 'Neighborhoods (old)', 'Police Districts',
       'Supervisor Districts', 'Fire Prevention Districts', 'Zip Codes',
       'Analysis Neighborhoods'],
      dtype='object')

In [3]:
df.head()

,business_id,business_name,business_address,business_city,business_state,business_postal_code,business_latitude,business_longitude,business_location,business_phone_number,...,inspection_type,violation_id,violation_description,risk_category,Neighborhoods (old),Police Districts,Supervisor Districts,Fire Prevention Districts,Zip Codes,Analysis Neighborhoods
0,101192,Cochinita #2,2 Marina Blvd Fort Mason,San Francisco,CA,NaN,NaN,NaN,NaN,1.415043e+10,...,New Ownership,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,97975,BREADBELLY,1408 Clement St,San Francisco,CA,94118,NaN,NaN,NaN,1.415724e+10,...,Routine - Unscheduled,97975_20190725_103124,Inadequately cleaned or sanitized food contact...,Moderate Risk,NaN,NaN,NaN,NaN,NaN,NaN
2,92982,Great Gold Restaurant,3161 24th St.,San Francisco,CA,94110,NaN,NaN,NaN,NaN,...,New Ownership,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,101389,HOMAGE,214 CALIFORNIA ST,San Francisco,CA,94111,NaN,NaN,NaN,1.415488e+10,...,New Construction,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,85986,Pronto Pizza,798 Eddy St,San Francisco,CA,94109,NaN,NaN,NaN,NaN,...,New Ownership,85986_20161011_103114,High risk vermin infestation,High Risk,NaN,NaN,NaN,NaN,NaN,NaN


#### Будем предсказывать оценку инспекции

In [5]:
df.dropna(inplace = True)

In [6]:
df.drop("business_address", axis=1, inplace = True)
df.drop("business_location", axis=1, inplace = True)
df.drop("business_phone_number", axis=1, inplace = True)

In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 6566 entries, 11 to 53854
Data columns (total 20 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   business_id                6566 non-null   int64  
 1   business_name              6566 non-null   object 
 2   business_city              6566 non-null   int64  
 3   business_state             6566 non-null   int64  
 4   business_postal_code       6566 non-null   int64  
 5   business_latitude          6566 non-null   float64
 6   business_longitude         6566 non-null   float64
 7   inspection_id              6566 non-null   object 
 8   inspection_date            6566 non-null   object 
 9   inspection_score           6566 non-null   float64
 10  inspection_type            6566 non-null   int64  
 11  violation_id               6566 non-null   object 
 12  violation_description      6566 non-null   object 
 13  risk_category              6566 non-null   int64  


In [8]:
label_encoder = LabelEncoder()
df["business_city"] = label_encoder.fit_transform(df["business_city"])
df["business_state"] = label_encoder.fit_transform(df["business_state"])
df["business_postal_code"] = label_encoder.fit_transform(df["business_postal_code"])
df["inspection_type"] = label_encoder.fit_transform(df["inspection_type"])
df["risk_category"] = label_encoder.fit_transform(df["risk_category"])

In [13]:
df.drop("business_name", axis=1, inplace = True)
df.drop("inspection_id", axis=1, inplace = True)
df.drop("inspection_date", axis=1, inplace = True)
df.drop("violation_id", axis=1, inplace = True)
df.drop("violation_description", axis=1, inplace = True)

In [14]:
df.describe()

,business_id,business_city,business_state,business_postal_code,business_latitude,business_longitude,inspection_score,inspection_type,risk_category,Neighborhoods (old),Police Districts,Supervisor Districts,Fire Prevention Districts,Zip Codes,Analysis Neighborhoods
count,6566.000000,6566.0,6566.0,6566.000000,6566.000000,6566.000000,6566.000000,6566.0,6566.00000,6566.000000,6566.000000,6566.000000,6566.000000,6566.000000,6566.000000
mean,17493.410600,0.0,0.0,11.442431,37.763030,-122.429826,85.840238,0.0,1.23180,19.629455,5.384861,6.671185,7.641182,20517.236065,19.961925
std,24297.938715,0.0,0.0,6.856196,0.023712,0.024573,8.247628,0.0,0.67613,10.791533,2.573118,2.955808,4.619384,13123.196468,11.036769
min,54.000000,0.0,0.0,0.000000,37.708887,-122.510043,46.000000,0.0,0.00000,1.000000,1.000000,1.000000,1.000000,54.000000,1.000000
25%,2397.000000,0.0,0.0,7.000000,37.748855,-122.437815,81.000000,0.0,1.00000,11.000000,4.000000,5.000000,2.000000,64.000000,11.000000
50%,4804.000000,0.0,0.0,11.000000,37.764814,-122.422035,87.000000,0.0,1.00000,19.000000,5.000000,7.000000,8.000000,28858.000000,20.000000
75%,21982.000000,0.0,0.0,17.000000,37.783257,-122.415109,92.000000,0.0,2.00000,27.000000,8.000000,9.000000,12.000000,28859.000000,29.000000
max,71909.000000,0.0,0.0,26.000000,37.808494,-122.380619,98.000000,0.0,2.00000,41.000000,10.000000,11.000000,15.000000,29492.000000,41.000000


In [15]:
df.size

98490

In [37]:
# Проведем масштабирование данных
from sklearn.preprocessing import MinMaxScaler, StandardScaler, Normalizer
sc2 = StandardScaler()
sc2_data = sc2.fit_transform(df)
sc2_data = pd.DataFrame(sc2_data, columns=df.columns)

In [43]:
X = sc2_data.drop(columns=['inspection_score'])
y = sc2_data['inspection_score']

In [44]:
from sklearn.preprocessing import MinMaxScaler

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)     

In [45]:
# Модели
dt_reg = DecisionTreeRegressor(random_state=0)
rf_reg = RandomForestRegressor(n_estimators=1000, random_state=0)

# Обучение моделей
dt_reg.fit(X_train, y_train)
rf_reg.fit(X_train, y_train)

# Предсказания моделей
y_pred_dt = dt_reg.predict(X_test)
y_pred_rf = rf_reg.predict(X_test)

In [46]:
dt_r2 = r2_score(y_test, y_pred_dt)
rf_r2 = r2_score(y_test, y_pred_rf)
dt_mse = mean_squared_error(y_test, y_pred_dt, squared=False)
rf_mse = mean_squared_error(y_test, y_pred_rf, squared=False)

In [47]:
print(f'Дерево решений R^2: {dt_r2:.4f}')
print(f'Случайное дерево R^2: {rf_r2:.4f}')
print()
print(f'Дерево решений MSE: {dt_mse:.4f}')
print(f'Случайное дерево MSE: {rf_mse:.4f}')

Дерево решений R^2: 0.6707
Случайное дерево R^2: 0.6954

Дерево решений MSE: 0.5929
Случайное дерево MSE: 0.5703


Я использовал две метрики: 
- R2 для определения процента данных, объясняемых моделью, а не стохаистически
- MSE для определения квадрата отклонения предсказаний

Набор входных признаков достаточно велик, поэтому вполне ожидаемо, что RandomForest выдал лучшие метрики, так как деревья решений очень чувствительны к признакам, по которым происходит ветвление.